In [ ]:
  # Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import os
import numpy as np

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
#Define image transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Step 2: Define Custom Dataset
class DeepfakeImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = {'Real': 0, 'Fake': 1}
        self.data = []

        for label in self.classes:
            class_dir = os.path.join(root_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.data.append((img_path, self.classes[label]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Install required packages
!pip install torch torchvision seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

train_dataset = DeepfakeImageDataset('/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Train', transform=transform)
val_dataset = DeepfakeImageDataset('/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation', transform=transform)
test_dataset = DeepfakeImageDataset('/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
#Define ResNeXt Model
class ResNeXtClassifier(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(ResNeXtClassifier, self).__init__()
        self.resnext = models.resnext50_32x4d(pretrained=True)
        num_ftrs = self.resnext.fc.in_features
        self.resnext.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_ftrs, 2)  # Adjust the final layer to output 2 classes
        )

    def forward(self, x):
        return self.resnext(x)

model = ResNeXtClassifier(dropout_rate=0.6).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 136MB/s]


In [ ]:
# Step 5: Define Loss Function, Optimizer, and Early Stopping Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
num_epochs = 10
patience = 3  # For early stopping
best_val_loss = np.inf
early_stop_counter = 0

In [ ]:
# Training Loop with Early Stopping
for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(labels).sum().item()

    train_accuracy = train_correct / len(train_dataset)
    train_loss /= len(train_loader)

    # Validation phase
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = val_correct / len(val_dataset)
    val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '/content/best_resnet_model.pth')
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            break


Epoch [1/10], Train Loss: 0.2587, Train Accuracy: 0.8796, Val Loss: 0.1293, Val Accuracy: 0.9494
Epoch [2/10], Train Loss: 0.1354, Train Accuracy: 0.9469, Val Loss: 0.2214, Val Accuracy: 0.9091
Epoch [3/10], Train Loss: 0.1155, Train Accuracy: 0.9584, Val Loss: 0.2780, Val Accuracy: 0.9033
Epoch [4/10], Train Loss: 0.1011, Train Accuracy: 0.9613, Val Loss: 0.1672, Val Accuracy: 0.9369
Early stopping triggered.


In [ ]:
pip install torch torchvision torchaudio

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/DL/best_resnext_model.pth')


In [ ]:
torch.save(model, '/content/drive/MyDrive/DL/entire_resnext_model.pth')


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/DL/best_resnext_model.pth'))

<ipython-input-10-f5d5fed428ed>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/DL/best_resnext_model.pth'))


<All keys matched successfully>

In [ ]:
from PIL import Image
import torch

def test_image(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    # Get prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    # Interpret the result
    class_labels = {0: 'Real', 1: 'Fake'}
    print(f"Predicted: {class_labels[predicted.item()]}")

# Example usage with your specific image path
test_image('/content/drive/MyDrive/Deep_learning/TrimDataset/Dataset/Validation/Fake/fake_2.jpg', model)


Predicted: Fake
